In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# New Section

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import cv2 
import random
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
data_dir = "/content/drive/MyDrive/BreastDataset"
data_dir = Path(data_dir)  
categories = os.listdir(data_dir)
categories.remove("InSitu")
categories.remove(".ipynb_checkpoints")
categories.remove("127.0.0.1_5000 - Google Chrome 2021-04-04 14-40-05.mp4")

In [ ]:
count_image = len(list(data_dir.glob("*/*.tif")))
print(count_image)
print(len(categories))
print(categories)

1150
3
['Normal', 'Invasive', 'Benign']


In [ ]:
training_data = []  # Toàn bộ dữ liệu chúng ta có thể sử dụng, phân biệt giữa training data, x_train, y_train, x_test, y_test, x_val và y_val
# training data = [x_train, y_train] + [x_test, y_test] + [x_val, y_val]

IMG_SIZE = (1000, 750)  # Trong hướng dẫn này, lựa chọn đưa toàn bộ ảnh về cùng kích thước 200 x 200, (bộ dữ liệu với nhiều loại kích thước ảnh)
def create_training_data():
  for category in categories: # Duyệt qua từng folder label
    path = os.path.join(data_dir, category) # tạo một path tạm đến folder label đó 
    label_number = categories.index(category) # label thứ mấy trong mảng categories chứa các label
    for img in os.listdir(path):  # duyệt qua toàn bộ ảnh trong path
      try:
        img_array_gray = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR) # Đọc mảng giá trị của ảnh, ở đây không chuyển ảnh thành gray
        new_img_array = cv2.resize(img_array_gray, IMG_SIZE, interpolation = cv2.INTER_AREA)  # Chuyển ảnh về cùng kích thước
        training_data.append([new_img_array, label_number]) # Lưu ảnh cùng label tương ứng vào training_data
      except Exception as e:
        pass

create_training_data()

In [ ]:
random.shuffle(training_data) # xáo trộn dữ liệu

In [ ]:
# Tạo 2 mảng để lưu samples và labels theo cùng index tương ứng
X = []  # samples
y = []  # labels

In [ ]:
for feature, label in training_data:  # Mỗi phần tử trong training_data có dạng [feature, label] trong đó feature = new_img_array và label = label_number
  X.append(feature)
  y.append(label)

# X bây giờ chứa toàn bộ features
# Chuyển X về dạng ma trận
# -1 ở đây tương đương với số lượng features, dùng -1 numpy tự hiểu là ứng với số lượng của feature hiện có trong X
# Vì không chuyển thành ảnh trắng đen nên ảnh có 3 kênh màu, tham số cuối cùng trong reshape
X = np.array(X).reshape(-1, IMG_SIZE[1], IMG_SIZE[0], 3)

In [ ]:
# Bước này dùng để lưu X và y vào một file theo định dạng có cấu trúc để khỏi phải chạy lại hàm tạo X, y phía trên - khá tốn thời gian cho việc tạo lại từ đầu
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("/content/drive/MyDrive/Colab Notebooks/y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
# Đọc lại file cấu trúc đã lưu
pickle_in = open("/content/drive/MyDrive/Colab Notebooks/X_4.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("/content/drive/MyDrive/Colab Notebooks/y_4.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
x_train = x_train/255 
x_test = x_test/255
x_val = x_val/225

y_train = to_categorical(y_train) # one hot
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

# New Section

In [ ]:
model = Sequential() #  Khởi tạo một một mô hình neural

# Bắt đầu của mạng thêm một Convolutional layer với các tham số trên
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# Lớp ban đầu này phải truyền cho mạng biết input_shape ban đầu là bao nhiêu chính là (weight x heigh x depth) của ảnh
# padding chỉ có thể chọn 1 trong 2 giá trị `same` hoặc `valid`
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='tanh', input_shape=X.shape[1:], padding="same"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='tanh', padding="same"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# Làm phẳng neuron, bắt đầu fully connected
model.add(Flatten())
# Tạo một layer
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

# output
model.add(Dense(len(categories), activation='softmax'))

# Cung cấp cho mô hình hàm loss function lựa chọn, thuật toán tối ưu
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary() # xem chi tiết mô hình xây dựng của chính mình

# Cuối cùng đào tạo nó
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 384, 512, 16)      448       
_________________________________________________________________
average_pooling2d (AveragePo (None, 192, 256, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 192, 256, 32)      4640      
_________________________________________________________________
average_pooling2d_1 (Average (None, 96, 128, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 128, 64)       18496     
_________________________________________________________________
average_pooling2d_2 (Average (None, 48, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 64, 64)        3

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

7/7 - 14s - loss: 0.9977 - accuracy: 0.5688
Restored model, accuracy: 56.88%
